In [63]:
import collections

# import helper
import numpy as np
# import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [65]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8861276655455413273
]


In [66]:
import pandas as pd

In [67]:
def load_data(path):
    with open(path, 'r') as fd:
      return fd.readlines()

In [68]:
df=pd.read_csv('processed_title.csv')

In [69]:
df

,Original,Target,Match
0,Delete all elements at application close,Delete all elements at application close in ASP,False
1,Which one of thesecode samples has better perf...,Which one of these code samples has better per...,False
2,Why library name gets an additional UNK NUMBER...,Why library name gets an additional UNK NUMBER...,False
3,What have you use Regular Expressions for,What have you used Regular Expressions for,False
4,Calculation of cubic bezier with known halfway...,Calculation of cubic B zier with known halfway...,False
...,...,...,...
22610,How to refresh a webpage automaticly every spe...,How to refresh a web page automatically every ...,False
22611,Help me to complete my UITableView to Navigati...,Help me to complete my UITableView to Navigati...,False
22612,JQuery focus() on jQuery-applied anchors works...,JQuery focus() on jQuery-applied anchors works...,False
22613,(tcl/Expect) clear screen after exit,(Tcl/Expect) clear screen after exit,False


In [80]:
df1=df

In [81]:
all_column_to_list=df1['Original'].tolist()+df1['Target'].tolist()

In [84]:
preprocess_x, x_tk = tokenize(all_column_to_list)

In [71]:
english_sentences=df1['Original']
french_sentences=df1['Target']

In [85]:
def tokenize_toseq(x,Tok):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    
#     tokenizer = Tokenizer()
    Tok.fit_on_texts(x)
#     print(Tok.texts_to_sequences(x))
    return Tok.texts_to_sequences(x)




In [72]:
english_sentences

0                 Delete all elements at application close
1        Which one of thesecode samples has better perf...
2        Why library name gets an additional UNK NUMBER...
3                What have you use Regular Expressions for
4        Calculation of cubic bezier with known halfway...
                               ...                        
22610    How to refresh a webpage automaticly every spe...
22611    Help me to complete my UITableView to Navigati...
22612    JQuery focus() on jQuery-applied anchors works...
22613                 (tcl/Expect) clear screen after exit
22614    How to call C# method taking an object of type...
Name: Original, Length: 22615, dtype: object

In [73]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

210794 English words.
26489 unique English words.
10 Most common words in the English dataset:
"to" "a" "in" "How" "the" "of" "and" "for" "NUMBER" "with"

216743 French words.
21989 unique French words.
10 Most common words in the French dataset:
"to" "a" "in" "How" "the" "of" "I" "and" "NUMBER" "for"


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    
    #Tokenizer(num_words=None, char_level=True)
    tokenizer = Tokenizer(num_words=None, char_level=True)
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

In [83]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
#     print("xxx")
#     print(x)
#     print("xxx")
    tokenizer = Tokenizer(num_words=None, char_level=True,filters='',lower=False)
#     tokenizer = Tokenizer(filters='',lower=False)
    tokenizer.fit_on_texts(x)
#     print(tokenizer.texts_to_sequences(x))
    return tokenizer.texts_to_sequences(x), tokenizer

test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{' ': 1, 'e': 2, 'o': 3, 'i': 4, 's': 5, 'h': 6, 'r': 7, 'y': 8, 'u': 9, 'c': 10, 'n': 11, 't': 12, 'a': 13, 'p': 14, '.': 15, 'T': 16, 'q': 17, 'k': 18, 'w': 19, 'f': 20, 'x': 21, 'm': 22, 'v': 23, 'l': 24, 'z': 25, 'd': 26, 'g': 27, 'b': 28, 'j': 29, 'B': 30, 'J': 31, ',': 32}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [16, 6, 2, 1, 17, 9, 4, 10, 18, 1, 28, 7, 3, 19, 11, 1, 20, 3, 21, 1, 29, 9, 22, 14, 5, 1, 3, 23, 2, 7, 1, 12, 6, 2, 1, 24, 13, 25, 8, 1, 26, 3, 27, 1, 15]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [30, 8, 1, 31, 3, 23, 2, 1, 32, 1, 22, 8, 1, 17, 9, 4, 10, 18, 1, 5, 12, 9, 26, 8, 1, 3, 20, 1, 24, 2, 21, 4, 10, 3, 27, 7, 13, 14, 6, 8, 1, 19, 3, 11, 1, 13, 1, 14, 7, 4, 25, 2, 1, 15]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [16, 6, 4, 5, 1, 4, 5, 1, 13, 1, 5, 6, 3, 7, 12, 1, 5, 2, 11, 12, 2, 11, 10, 2, 1, 15]


In [76]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

# test_pad(pad)

# # Pad Tokenized output
# test_pad = pad(text_tokenized)
# for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
#     print('Sequence {} in x'.format(sample_i + 1))
#     print('  Input:  {}'.format(np.array(token_sent)))
#     print('  Output: {}'.format(pad_sent))

In [77]:
len(english_sentences)

22615

In [78]:
english_sentences

0                 Delete all elements at application close
1        Which one of thesecode samples has better perf...
2        Why library name gets an additional UNK NUMBER...
3                What have you use Regular Expressions for
4        Calculation of cubic bezier with known halfway...
                               ...                        
22610    How to refresh a webpage automaticly every spe...
22611    Help me to complete my UITableView to Navigati...
22612    JQuery focus() on jQuery-applied anchors works...
22613                 (tcl/Expect) clear screen after exit
22614    How to call C# method taking an object of type...
Name: Original, Length: 22615, dtype: object

In [86]:
def preprocess1(x,y,x_tk):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x = tokenize_toseq(x,x_tk)
    preprocess_y = tokenize_toseq(y,x_tk)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    return preprocess_x, preprocess_y, x_tk, x_tk

In [87]:

preproc_english_sentences1, preproc_french_sentences1, english_tokenizer1, french_tokenizer =\
    preprocess1(english_sentences, french_sentences,x_tk)

In [79]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    print("???????????????")
    print(x)
    print("???????????????")
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

???????????????
0                 Delete all elements at application close
1        Which one of thesecode samples has better perf...
2        Why library name gets an additional UNK NUMBER...
3                What have you use Regular Expressions for
4        Calculation of cubic bezier with known halfway...
                               ...                        
22610    How to refresh a webpage automaticly every spe...
22611    Help me to complete my UITableView to Navigati...
22612    JQuery focus() on jQuery-applied anchors works...
22613                 (tcl/Expect) clear screen after exit
22614    How to call C# method taking an object of type...
Name: Original, Length: 22615, dtype: object
???????????????
xxx
0                 Delete all elements at application close
1        Which one of thesecode samples has better perf...
2        Why library name gets an additional UNK NUMBER...
3                What have you use Regular Expressions for
4        Calculation of cubic bezi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Data Preprocessed
Max English sentence length: 183
Max French sentence length: 180
English vocabulary size: 79
French vocabulary size: 78


In [88]:
import pandas as pd
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense,  RepeatVector, TimeDistributed
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [19]:
french_sentences

0          Delete all elements at application close in ASP
1        Which one of these code samples has better per...
2        Why library name gets an additional UNK NUMBER...
3               What have you used Regular Expressions for
4        Calculation of cubic B zier with known halfway...
                               ...                        
22610    How to refresh a web page automatically every ...
22611    Help me to complete my UITableView to Navigati...
22612    JQuery focus() on jQuery-applied anchors works...
22613                 (Tcl/Expect) clear screen after exit
22614    How to call C# method taking a parameter of ty...
Name: Target, Length: 22615, dtype: object

In [20]:
english_tokenizer.index_word

{1: ' ',
 2: 'e',
 3: 't',
 4: 'o',
 5: 'a',
 6: 'i',
 7: 'n',
 8: 'r',
 9: 's',
 10: 'l',
 11: 'c',
 12: 'd',
 13: 'h',
 14: 'u',
 15: 'p',
 16: 'm',
 17: 'g',
 18: 'w',
 19: 'f',
 20: 'y',
 21: 'b',
 22: 'v',
 23: 'N',
 24: 'S',
 25: 'U',
 26: 'M',
 27: 'I',
 28: 'C',
 29: 'H',
 30: 'E',
 31: 'R',
 32: 'k',
 33: 'P',
 34: 'B',
 35: 'x',
 36: 'L',
 37: 'W',
 38: 'A',
 39: 'D',
 40: 'K',
 41: 'T',
 42: 'j',
 43: '-',
 44: 'q',
 45: 'F',
 46: 'Q',
 47: "'",
 48: 'O',
 49: ':',
 50: '(',
 51: ',',
 52: 'V',
 53: 'J',
 54: ')',
 55: '+',
 56: '"',
 57: 'G',
 58: '/',
 59: '#',
 60: 'z',
 61: 'X',
 62: ']',
 63: '[',
 64: '_',
 65: '>',
 66: '<',
 67: 'Y',
 68: '*',
 69: '&',
 70: 'Z',
 71: '$',
 72: '`',
 73: '@',
 74: '\\',
 75: '%',
 76: '|',
 77: '~',
 78: '^',
 79: '\x7f'}

def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

In [21]:
import pandas as pd
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense,  RepeatVector, TimeDistributed
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [22]:
preproc_english_sentences

array([[39,  2, 10, ...,  0,  0,  0],
       [37, 13,  6, ...,  0,  0,  0],
       [37, 13, 20, ...,  0,  0,  0],
       ...,
       [53, 46, 14, ...,  0,  0,  0],
       [50,  3, 11, ...,  0,  0,  0],
       [29,  4, 18, ...,  0,  0,  0]])

In [89]:
tmp_x = tf.keras.utils.to_categorical(preproc_english_sentences1)
tmp_y = tf.keras.utils.to_categorical(preproc_french_sentences1)

In [90]:
max_english_sequence_length = preproc_english_sentences1.shape[1]
max_french_sequence_length = preproc_french_sentences1.shape[1]
english_vocab_size = len(english_tokenizer1.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 183
Max French sentence length: 180
English vocabulary size: 79
French vocabulary size: 79


In [24]:
df1

,Original,Target,Match
0,Delete all elements at application close,Delete all elements at application close in ASP,False
1,Which one of thesecode samples has better perf...,Which one of these code samples has better per...,False
2,Why library name gets an additional UNK NUMBER...,Why library name gets an additional UNK NUMBER...,False
3,What have you use Regular Expressions for,What have you used Regular Expressions for,False
4,Calculation of cubic bezier with known halfway...,Calculation of cubic B zier with known halfway...,False
...,...,...,...
22610,How to refresh a webpage automaticly every spe...,How to refresh a web page automatically every ...,False
22611,Help me to complete my UITableView to Navigati...,Help me to complete my UITableView to Navigati...,False
22612,JQuery focus() on jQuery-applied anchors works...,JQuery focus() on jQuery-applied anchors works...,False
22613,(tcl/Expect) clear screen after exit,(Tcl/Expect) clear screen after exit,False


In [56]:
preproc_english_sentences1, preproc_french_sentences1, english_tokenizer1, french_tokenizer

In [92]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model



# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences1, preproc_french_sentences1.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences1.shape[-2]))

# TODO: Train the neural network
embed_rnn_model1 = embed_model(
    tmp_x.shape,
    preproc_french_sentences1.shape[1],
    len(english_tokenizer1.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model1.summary()

embed_rnn_model1.fit(tmp_x, preproc_french_sentences1, batch_size=1024, epochs=10, validation_split=0.2)

# # TODO: Print prediction(s)
# print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 180, 256)          20480     
_________________________________________________________________
gru_2 (GRU)                  (None, 180, 256)          394752    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 180, 1024)         263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, 180, 1024)         0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 180, 80)           82000     
Total params: 760,400
Trainable params: 760,400
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
18/18 [==============================] - 69s 4s/step - loss: 1.6770 - accuracy: 0.6838 - val_loss: 1.0

In [52]:
import collections

# import helper
import numpy as np
# import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [53]:
import pandas as pd
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense,  RepeatVector, TimeDistributed
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [54]:
from keras.models import Sequential
from keras.layers import Activation, SimpleRNN

In [55]:
import pandas as pd
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11176845779567890634
]


def encdec_model(input_shape, output_sequence_length, original_vocab_size, target_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param original_vocab_size: Number of unique character in original text
    :param target_vocab_size: Number of unique character in target text
    :return: Keras model built, but not trained
    """    
    # Hyperparameters
    learning_rate = 0.001
    
    model = Sequential()
    model.add(LSTM(256, input_shape=input_shape))
    model.add(RepeatVector(output_sequence_length))
    model.add(SimpleRNN(1024, return_sequences=True))
    model.add(Dense(1024, activation='relu'))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(target_vocab_size, activation='softmax')))

    model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model

    
    return model

In [ ]:
preproc_english_sentences1, preproc_french_sentences1, english_tokenizer1, french_tokenizer

In [62]:
# TODO: Reshape the input
# tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# Embedding the dataset
# tmp_x = tf.keras.utils.to_categorical(preproc_original_sentences)
# tmp_y = tf.keras.utils.to_categorical(preproc_target_sentences)

# TODO: Train the neural network
embed_rnn_model1 = encdec_model(
    tmp_x.shape[1],
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model1.summary()

embed_rnn_model1.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

TypeError: 'int' object is not iterable

In [32]:
len(tmp_x)

22615

here is the training

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model



# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# # TODO: Print prediction(s)
# print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

In [33]:
embed_rnn_model1.save('my_model_with_all_char.h5') 

In [34]:
from keras.models import load_model


model = load_model('my_model_with_all_char.h5')

In [ ]:
model.

In [35]:
len(tmp_x)

22615

In [36]:
xer=df['Original'][1]

In [37]:
xer

'Which one of thesecode samples has better performance'

In [38]:
tokenize(xer)

xxx
Which one of thesecode samples has better performance
xxx
[[14], [3], [15], [6], [3], [2], [4], [10], [1], [2], [4], [11], [2], [7], [3], [1], [5], [1], [6], [4], [16], [1], [2], [5], [8], [12], [13], [17], [1], [5], [2], [3], [8], [5], [2], [18], [1], [7], [7], [1], [9], [2], [13], [1], [9], [11], [4], [9], [12], [8], [10], [6], [1]]


([[14],
  [3],
  [15],
  [6],
  [3],
  [2],
  [4],
  [10],
  [1],
  [2],
  [4],
  [11],
  [2],
  [7],
  [3],
  [1],
  [5],
  [1],
  [6],
  [4],
  [16],
  [1],
  [2],
  [5],
  [8],
  [12],
  [13],
  [17],
  [1],
  [5],
  [2],
  [3],
  [8],
  [5],
  [2],
  [18],
  [1],
  [7],
  [7],
  [1],
  [9],
  [2],
  [13],
  [1],
  [9],
  [11],
  [4],
  [9],
  [12],
  [8],
  [10],
  [6],
  [1]],
 <keras_preprocessing.text.Tokenizer at 0x1ad78cbf9a0>)

In [39]:
df

,Original,Target,Match
0,Delete all elements at application close,Delete all elements at application close in ASP,False
1,Which one of thesecode samples has better perf...,Which one of these code samples has better per...,False
2,Why library name gets an additional UNK NUMBER...,Why library name gets an additional UNK NUMBER...,False
3,What have you use Regular Expressions for,What have you used Regular Expressions for,False
4,Calculation of cubic bezier with known halfway...,Calculation of cubic B zier with known halfway...,False
...,...,...,...
22610,How to refresh a webpage automaticly every spe...,How to refresh a web page automatically every ...,False
22611,Help me to complete my UITableView to Navigati...,Help me to complete my UITableView to Navigati...,False
22612,JQuery focus() on jQuery-applied anchors works...,JQuery focus() on jQuery-applied anchors works...,False
22613,(tcl/Expect) clear screen after exit,(Tcl/Expect) clear screen after exit,False


In [93]:
import tokenize

import pandas as pd
import ast
import astor
import pandas as pd
import os
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
def token_create(code):
    f = code.replace("\n", " ")
    
    
    data = []
    
    # iterate through each sentence in the file
    for i in sent_tokenize(f):
        temp = []

        # tokenize the sentence into words
        for j in word_tokenize(i):
            temp.append(j)

        data.append(temp)
#         print('done')
        
        
      
    return data 

In [94]:
x=token_create(df['Original'][8])

In [95]:
x=x[0]

In [96]:
english_tokenizer.index_word

{1: ' ',
 2: 'e',
 3: 't',
 4: 'o',
 5: 'a',
 6: 'i',
 7: 'n',
 8: 'r',
 9: 's',
 10: 'l',
 11: 'c',
 12: 'd',
 13: 'h',
 14: 'u',
 15: 'p',
 16: 'm',
 17: 'g',
 18: 'w',
 19: 'f',
 20: 'y',
 21: 'b',
 22: 'v',
 23: 'N',
 24: 'S',
 25: 'U',
 26: 'M',
 27: 'I',
 28: 'C',
 29: 'H',
 30: 'E',
 31: 'R',
 32: 'k',
 33: 'P',
 34: 'B',
 35: 'x',
 36: 'L',
 37: 'W',
 38: 'A',
 39: 'D',
 40: 'K',
 41: 'T',
 42: 'j',
 43: '-',
 44: 'q',
 45: 'F',
 46: 'Q',
 47: "'",
 48: 'O',
 49: ':',
 50: '(',
 51: ',',
 52: 'V',
 53: 'J',
 54: ')',
 55: '+',
 56: '"',
 57: 'G',
 58: '/',
 59: '#',
 60: 'z',
 61: 'X',
 62: ']',
 63: '[',
 64: '_',
 65: '>',
 66: '<',
 67: 'Y',
 68: '*',
 69: '&',
 70: 'Z',
 71: '$',
 72: '`',
 73: '@',
 74: '\\',
 75: '%',
 76: '|',
 77: '~',
 78: '^',
 79: '\x7f'}

In [97]:
preproc_french_sentences

array([[[39],
        [ 2],
        [10],
        ...,
        [ 0],
        [ 0],
        [ 0]],

       [[37],
        [13],
        [ 6],
        ...,
        [ 0],
        [ 0],
        [ 0]],

       [[37],
        [13],
        [20],
        ...,
        [ 0],
        [ 0],
        [ 0]],

       ...,

       [[48],
        [44],
        [14],
        ...,
        [ 0],
        [ 0],
        [ 0]],

       [[50],
        [40],
        [11],
        ...,
        [ 0],
        [ 0],
        [ 0]],

       [[27],
        [ 4],
        [18],
        ...,
        [ 0],
        [ 0],
        [ 0]]])

In [98]:
sentence

NameError: name 'sentence' is not defined

In [99]:
xp=df1['Original'][2]

In [100]:
xp

'Why library name gets an additional UNK NUMBER is its name'

In [101]:
sentence=xp.lower()

In [102]:

    
sentence = [english_tokenizer.word_index[ch] for ch in sentence]


#     sentence = [x_tk.word_index[word] for word in sentence.split()]
print("??????????")
print(sentence)
print("??????????")
sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
sentences = np.array([sentence[0], preproc_english_sentences[0]])
predictions = embed_rnn_model1.predict(sentences, len(sentences))
print("??????????")
print(predictions)

??????????
[18, 13, 20, 1, 10, 6, 21, 8, 5, 8, 20, 1, 7, 5, 16, 2, 1, 17, 2, 3, 9, 1, 5, 7, 1, 5, 12, 12, 6, 3, 6, 4, 7, 5, 10, 1, 14, 7, 32, 1, 7, 14, 16, 21, 2, 8, 1, 6, 9, 1, 6, 3, 9, 1, 7, 5, 16, 2]
??????????
??????????
[[[7.57400319e-03 1.14331432e-02 1.63492057e-02 ... 2.97036750e-04
   3.13405268e-04 2.17293811e-04]
  [3.81068140e-03 2.43498031e-02 4.65650931e-02 ... 2.69329248e-05
   2.60838060e-05 1.38344167e-05]
  [3.18638328e-03 9.43890959e-02 1.05332270e-01 ... 3.45258159e-05
   2.11147490e-05 1.05449826e-05]
  ...
  [9.99999166e-01 8.22964665e-08 1.93553646e-07 ... 2.05383324e-20
   1.92056394e-20 5.26920997e-21]
  [9.99999166e-01 8.09367151e-08 1.90840396e-07 ... 1.93944822e-20
   1.81391257e-20 4.96789772e-21]
  [9.99999166e-01 7.96287338e-08 1.88226394e-07 ... 1.83371288e-20
   1.71529584e-20 4.68987658e-21]]

 [[3.43145477e-03 6.85256300e-03 9.64288507e-03 ... 7.26189974e-05
   7.82822390e-05 3.95696370e-05]
  [1.93253043e-03 3.42112556e-02 9.50319469e-02 ... 1.060193

In [103]:
def logits_to_text(logits,tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
#     print(tokenizer.index_word.get(logits))
    r=tokenizer.index_word.get(logits)
    if r!=None:
        r=str(r)
    else:
        r=""
    return r

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [104]:
xer

NameError: name 'xer' is not defined

In [105]:
xp=df['Original'][0]

In [106]:
xp

'Delete all elements at application close'

In [107]:
tokenize(xp)

TypeError: 'module' object is not callable

In [108]:
sentence = xer1
sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
sentences = np.array([sentence[0], preproc_english_sentences[0]])
predictions = model.predict(sentences, len(sentences))


NameError: name 'xer1' is not defined

In [109]:
xr=df['Original'][8]

In [110]:
xr

'How to format a Date - Format String Question'

In [111]:
sentence=[english_tokenizer.word_index[ch] for ch in xr]

In [112]:
english_tokenizer.index_word

{1: ' ',
 2: 'e',
 3: 't',
 4: 'o',
 5: 'a',
 6: 'i',
 7: 'n',
 8: 'r',
 9: 's',
 10: 'l',
 11: 'c',
 12: 'd',
 13: 'h',
 14: 'u',
 15: 'p',
 16: 'm',
 17: 'g',
 18: 'w',
 19: 'f',
 20: 'y',
 21: 'b',
 22: 'v',
 23: 'N',
 24: 'S',
 25: 'U',
 26: 'M',
 27: 'I',
 28: 'C',
 29: 'H',
 30: 'E',
 31: 'R',
 32: 'k',
 33: 'P',
 34: 'B',
 35: 'x',
 36: 'L',
 37: 'W',
 38: 'A',
 39: 'D',
 40: 'K',
 41: 'T',
 42: 'j',
 43: '-',
 44: 'q',
 45: 'F',
 46: 'Q',
 47: "'",
 48: 'O',
 49: ':',
 50: '(',
 51: ',',
 52: 'V',
 53: 'J',
 54: ')',
 55: '+',
 56: '"',
 57: 'G',
 58: '/',
 59: '#',
 60: 'z',
 61: 'X',
 62: ']',
 63: '[',
 64: '_',
 65: '>',
 66: '<',
 67: 'Y',
 68: '*',
 69: '&',
 70: 'Z',
 71: '$',
 72: '`',
 73: '@',
 74: '\\',
 75: '%',
 76: '|',
 77: '~',
 78: '^',
 79: '\x7f'}

In [113]:
sentence

[29,
 4,
 18,
 1,
 3,
 4,
 1,
 19,
 4,
 8,
 16,
 5,
 3,
 1,
 5,
 1,
 39,
 5,
 3,
 2,
 1,
 43,
 1,
 45,
 4,
 8,
 16,
 5,
 3,
 1,
 24,
 3,
 8,
 6,
 7,
 17,
 1,
 46,
 14,
 2,
 9,
 3,
 6,
 4,
 7]

In [119]:
def sentence_pred(data):
    
#     print(data)
    sentence=[english_tokenizer.word_index[ch] for ch in data]
    oo=sentence
    print("???")
#     print(oo)
    print("???")
#     sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=preproc_english_sentences.shape[-1], padding='post')
    print(sentence)
    sentences = np.array([sentence[0], preproc_english_sentences[0]])
    print(sentences)
    predictions = embed_rnn_model1.predict(sentences, len(sentences))
    RX=""
    for prediction in np.argmax(predictions[0], 1):
    #     print(prediction)
        dc=logits_to_text(prediction,x_tk)
        dc=str(dc)
#         print(dc)
        if dc!=None:
#             RX+=dc+" "
            RX+=dc
#     print(RX) 
    sentence = " ".join(RX.split())
    return sentence

In [70]:
xr

'How to format a Date - Format String Question'

In [71]:
io=df['Original'][0]

In [ ]:
for ch in io:
    print(ch)
[english_tokenizer.word_index[ch] for ch in io]

In [58]:
df['Original'][200]

'Would this desgin be ok - Interfaces and Abstract class'

In [116]:
df1=df.head(50)

In [122]:
df1['Prediction_with_Symbol']=df1['Original'].apply(sentence_pred)

???
???
[[39  2 10  2  3  2  1  5 10 10  1  2 10  2 16  2  7  3  9  1  5  3  1  5
  15 15 10  6 11  5  3  6  4  7  1 11 10  4  9  2  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
[[39  2 10  2  3  2  1  5 10 10  1  2 10  2 16  2  7  3  9  1  5  3  1  5
  15 15 10  6 11  5  3  6  4  7  1 11 10  4  9  2  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

C:\Users\subra\AppData\Local\Temp\ipykernel_11208\1639125012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Prediction_with_Symbol']=df1['Original'].apply(sentence_pred)


In [123]:
df1

,Original,Target,Match,Prediction_with_Symbol
0,Delete all elements at application close,Delete all elements at application close in ASP,False,Hoveelete all elements at application clos
1,Which one of thesecode samples has better perf...,Which one of these code samples has better per...,False,Whhwhich one of thesecode samples aas better p...
2,Why library name gets an additional UNK NUMBER...,Why library name gets an additional UNK NUMBER...,False,Whwwhy library name gets an additional S UUBER...
3,What have you use Regular Expressions for,What have you used Regular Expressions for,False,Whhwhat have you use Regular Expresssons fo
4,Calculation of cubic bezier with known halfway...,Calculation of cubic B zier with known halfway...,False,Hhaaallulation of cubit bezier with noo halu a...
5,"How to ""Replace in files","How to multi-line ""Replace in files",False,Co Cow to Replace in filee
6,Can someone explain this c++ notation,Can someone explain this C++ notation,False,Hhaaan someone explain this C++ notatoo
7,Best way to get started on simple UNK NUMBERD ...,Best way to get started on simple UNK NUMBERD ...,False,Sherest way to get started on simple S UUBERD ...
8,How to format a Date - Format String Question,How to format a Date without using code - Form...,False,Co Cow to format a Date - Format rring Questin
9,can quantum algorithms be used for encryption,Can quantum algorithms be used for encryption,False,Whacan quantum algoritars be used for encrpptio


In [124]:
df

,Original,Target,Match
0,Delete all elements at application close,Delete all elements at application close in ASP,False
1,Which one of thesecode samples has better perf...,Which one of these code samples has better per...,False
2,Why library name gets an additional UNK NUMBER...,Why library name gets an additional UNK NUMBER...,False
3,What have you use Regular Expressions for,What have you used Regular Expressions for,False
4,Calculation of cubic bezier with known halfway...,Calculation of cubic B zier with known halfway...,False
...,...,...,...
22610,How to refresh a webpage automaticly every spe...,How to refresh a web page automatically every ...,False
22611,Help me to complete my UITableView to Navigati...,Help me to complete my UITableView to Navigati...,False
22612,JQuery focus() on jQuery-applied anchors works...,JQuery focus() on jQuery-applied anchors works...,False
22613,(tcl/Expect) clear screen after exit,(Tcl/Expect) clear screen after exit,False


In [37]:
result = df.to_json(orient="columns")

In [38]:
df.to_json(r'data.json')

In [125]:
df3=pd.r.0*-ead_json('data.json')

ValueError: Expected object or value

In [ ]:
df

In [108]:
from collections import Counter

from nltk.util import everygrams, ngrams

In [109]:
def sentence_gleu(references, hypothesis, min_len=1, max_len=40):
    
    return corpus_gleu([references], [hypothesis], min_len=min_len, max_len=max_len)

In [110]:
def corpus_gleu(list_of_references, hypotheses, min_len=1, max_len=4):
    """
    Calculate a single corpus-level GLEU score (aka. system-level GLEU) for all
    the hypotheses and their respective references.

    Instead of averaging the sentence level GLEU scores (i.e. macro-average
    precision), Wu et al. (2016) sum up the matching tokens and the max of
    hypothesis and reference tokens for each sentence, then compute using the
    aggregate values.

    From Mike Schuster (via email):
        "For the corpus, we just add up the two statistics n_match and
         n_all = max(n_all_output, n_all_target) for all sentences, then
         calculate gleu_score = n_match / n_all, so it is not just a mean of
         the sentence gleu scores (in our case, longer sentences count more,
         which I think makes sense as they are more difficult to translate)."

    >>> hyp1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
    ...         'ensures', 'that', 'the', 'military', 'always',
    ...         'obeys', 'the', 'commands', 'of', 'the', 'party']
    >>> ref1a = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
    ...          'ensures', 'that', 'the', 'military', 'will', 'forever',
    ...          'heed', 'Party', 'commands']
    >>> ref1b = ['It', 'is', 'the', 'guiding', 'principle', 'which',
    ...          'guarantees', 'the', 'military', 'forces', 'always',
    ...          'being', 'under', 'the', 'command', 'of', 'the', 'Party']
    >>> ref1c = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
    ...          'army', 'always', 'to', 'heed', 'the', 'directions',
    ...          'of', 'the', 'party']

    >>> hyp2 = ['he', 'read', 'the', 'book', 'because', 'he', 'was',
    ...         'interested', 'in', 'world', 'history']
    >>> ref2a = ['he', 'was', 'interested', 'in', 'world', 'history',
    ...          'because', 'he', 'read', 'the', 'book']

    >>> list_of_references = [[ref1a, ref1b, ref1c], [ref2a]]
    >>> hypotheses = [hyp1, hyp2]
    >>> corpus_gleu(list_of_references, hypotheses) # doctest: +ELLIPSIS
    0.5673...

    The example below show that corpus_gleu() is different from averaging
    sentence_gleu() for hypotheses

    >>> score1 = sentence_gleu([ref1a], hyp1)
    >>> score2 = sentence_gleu([ref2a], hyp2)
    >>> (score1 + score2) / 2 # doctest: +ELLIPSIS
    0.6144...

    :param list_of_references: a list of reference sentences, w.r.t. hypotheses
    :type list_of_references: list(list(list(str)))
    :param hypotheses: a list of hypothesis sentences
    :type hypotheses: list(list(str))
    :param min_len: The minimum order of n-gram this function should extract.
    :type min_len: int
    :param max_len: The maximum order of n-gram this function should extract.
    :type max_len: int
    :return: The corpus-level GLEU score.
    :rtype: float
    """
    # sanity check
    assert len(list_of_references) == len(
        hypotheses
    ), "The number of hypotheses and their reference(s) should be the same"

    # sum matches and max-token-lengths over all sentences
    corpus_n_match = 0
    corpus_n_all = 0

    for references, hypothesis in zip(list_of_references, hypotheses):
        hyp_ngrams = Counter(everygrams(hypothesis, min_len, max_len))
        tpfp = sum(hyp_ngrams.values())  # True positives + False positives.

        hyp_counts = []
        for reference in references:
            ref_ngrams = Counter(everygrams(reference, min_len, max_len))
            tpfn = sum(ref_ngrams.values())  # True positives + False negatives.

            overlap_ngrams = ref_ngrams & hyp_ngrams
            tp = sum(overlap_ngrams.values())  # True positives.

            # While GLEU is defined as the minimum of precision and
            # recall, we can reduce the number of division operations by one by
            # instead finding the maximum of the denominators for the precision
            # and recall formulae, since the numerators are the same:
            #     precision = tp / tpfp
            #     recall = tp / tpfn
            #     gleu_score = min(precision, recall) == tp / max(tpfp, tpfn)
            n_all = max(tpfp, tpfn)

            if n_all > 0:
                hyp_counts.append((tp, n_all))

        # use the reference yielding the highest score
        if hyp_counts:
            n_match, n_all = max(hyp_counts, key=lambda hc: hc[0] / hc[1])
            corpus_n_match += n_match
            corpus_n_all += n_all

    # corner case: empty corpus or empty references---don't divide by zero!
    if corpus_n_all == 0:
        gleu_score = 0.0
    else:
        gleu_score = corpus_n_match / corpus_n_all

    return gleu_score

In [134]:
>>> ref = 'the cat -is on the mat  dpk'.split()
>>> hyp = 'the cat is on the mat'.split()
>>> sentence_gleu([ref], hyp)  


0.4090909090909091